# Ventilator Train Transformer

![](https://user-images.githubusercontent.com/544269/27125686-0ccd1b82-5130-11e7-8004-44e357dc9b25.png)

In this notebook, I train the transformer model.

If you want to know Transformer, you can see [this paper](https://arxiv.org/abs/1706.03762).

In [ ]:
from kaggle_secrets import UserSecretsClient
secret_label = "wandb"
secret_value = UserSecretsClient().get_secret(secret_label)
!wandb login $secret_value

In [ ]:
import gc
import os
import random
import wandb

import numpy as np
import pandas as pd

from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

device = torch.device("cuda")

In [ ]:
class config:
    EXP_NAME = "exp010_transformer"
    
    INPUT = "/kaggle/input/ventilator-pressure-prediction"
    OUTPUT = "/kaggle/working"
    N_FOLD = 5
    SKIP_FOLDS = [1, 2, 3, 4]  # only fold-0
    SEED = 0
    
    LR = 2.5e-2
    N_EPOCHS = 50
    HIDDEN_SIZE = 64
    BS = 2048
    WEIGHT_DECAY = 1e-5
    
    NOT_WATCH_PARAM = ['INPUT']

In [ ]:
def set_seed(seed=config.SEED):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## Dataset Class

In [ ]:
class VentilatorDataset(Dataset):
    
    def __init__(self, df):
        self.dfs = [_df for _, _df in df.groupby("breath_id")]
        
    def __len__(self):
        return len(self.dfs)
    
    def __getitem__(self, item):
        df = self.dfs[item]
        
        X = df[['R_cate', 'C_cate', 'u_in', 'u_out']].values
        y = df['pressure'].values
        d = {
            "X": torch.tensor(X).float(),
            "y": torch.tensor(y).float(),
        }
        return d

## Transformer Model

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
class VentilatorModel(nn.Module):
    
    def __init__(self):
        super(VentilatorModel, self).__init__()
        # This embedding method from: https://www.kaggle.com/theoviel/deep-learning-starter-simple-lstm
        self.r_emb = nn.Embedding(3, 2, padding_idx=0)
        self.c_emb = nn.Embedding(3, 2, padding_idx=0)
        self.seq_emb = nn.Sequential(
            nn.Linear(9, config.HIDDEN_SIZE),
            nn.LayerNorm(config.HIDDEN_SIZE),
            nn.ReLU(),
            nn.Dropout(0.2),
        )
        self.pos_encoder = PositionalEncoding(d_model=config.HIDDEN_SIZE, dropout=0.2)
        encoder_layers = nn.TransformerEncoderLayer(d_model=config.HIDDEN_SIZE, nhead=8, dim_feedforward=2048, dropout=0.2, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=2)
        self.head = nn.Linear(config.HIDDEN_SIZE, 1)
        
        # Encoder
        initrange = 0.1
        self.r_emb.weight.data.uniform_(-initrange, initrange)
        self.c_emb.weight.data.uniform_(-initrange, initrange)

    def forward(self, X, y=None):
        bs = X.shape[0]
        r_emb = self.r_emb(X[:,:,0].long()).view(bs, 80, -1)
        c_emb = self.c_emb(X[:,:,1].long()).view(bs, 80, -1)
        seq_x = torch.cat((r_emb, c_emb, X[:, :, 2:]), 2)
        h = self.seq_emb(seq_x)
        h = self.pos_encoder(h)
        h = self.transformer_encoder(h)
        regr = self.head(h)
        
        if y is None:
            loss = None
        else:
            loss = self.loss_fn(regr.squeeze(2), y)
            
        return regr, loss
    
    def loss_fn(self, y_pred, y_true):
        loss = nn.L1Loss()(y_pred, y_true)
        return loss

In [ ]:
def train_loop(model, optimizer, loader):
    losses, lrs = [], []
    model.train()
    optimizer.zero_grad()
    for d in loader:
        out, loss = model(d['X'].to(device), d['y'].to(device))
        
        losses.append(loss.item())
        step_lr = np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean()
        lrs.append(step_lr)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return np.array(losses).mean(), np.array(lrs).mean()

def valid_loop(model, loader):
    losses, predicts = [], []
    model.eval()
    for d in loader:
        with torch.no_grad():
            out, loss = model(d['X'].to(device), d['y'].to(device))
        losses.append(loss.item())
        predicts.append(out.cpu())

    return np.array(losses).mean(), torch.vstack(predicts).squeeze(2).numpy().reshape(-1)

def test_loop(model, loader):
    predicts = []
    model.eval()
    for d in loader:
        with torch.no_grad():
            out, _ = model(d['X'].to(device))
        predicts.append(out.cpu())

    return torch.vstack(predicts).squeeze(2).numpy().reshape(-1)

## Optimizer

I use Lamb Optimizer. (cf: https://arxiv.org/abs/1904.00962)

This optimizer can treat large batch sizes efficiently.

In [ ]:
from torch.optim.optimizer import Optimizer
class Lamb(Optimizer):
    # Reference code: https://github.com/cybertronai/pytorch-lamb

    def __init__(
        self,
        params,
        lr: float = 1e-3,
        betas = (0.9, 0.999),
        eps: float = 1e-6,
        weight_decay: float = 0,
        clamp_value: float = 10,
        adam: bool = False,
        debias: bool = False,
    ):
        if lr <= 0.0:
            raise ValueError('Invalid learning rate: {}'.format(lr))
        if eps < 0.0:
            raise ValueError('Invalid epsilon value: {}'.format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError(
                'Invalid beta parameter at index 0: {}'.format(betas[0])
            )
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError(
                'Invalid beta parameter at index 1: {}'.format(betas[1])
            )
        if weight_decay < 0:
            raise ValueError(
                'Invalid weight_decay value: {}'.format(weight_decay)
            )
        if clamp_value < 0.0:
            raise ValueError('Invalid clamp value: {}'.format(clamp_value))

        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.clamp_value = clamp_value
        self.adam = adam
        self.debias = debias

        super(Lamb, self).__init__(params, defaults)

    def step(self, closure = None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    msg = (
                        'Lamb does not support sparse gradients, '
                        'please consider SparseAdam instead'
                    )
                    raise RuntimeError(msg)

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(
                        p, memory_format=torch.preserve_format
                    )
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(
                        p, memory_format=torch.preserve_format
                    )

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # Decay the first and second moment running average coefficient
                # m_t
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                # v_t
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                # Paper v3 does not use debiasing.
                if self.debias:
                    bias_correction = math.sqrt(1 - beta2 ** state['step'])
                    bias_correction /= 1 - beta1 ** state['step']
                else:
                    bias_correction = 1

                # Apply bias to lr to avoid broadcast.
                step_size = group['lr'] * bias_correction

                weight_norm = torch.norm(p.data).clamp(0, self.clamp_value)

                adam_step = exp_avg / exp_avg_sq.sqrt().add(group['eps'])
                if group['weight_decay'] != 0:
                    adam_step.add_(p.data, alpha=group['weight_decay'])

                adam_norm = torch.norm(adam_step)
                if weight_norm == 0 or adam_norm == 0:
                    trust_ratio = 1
                else:
                    trust_ratio = weight_norm / adam_norm
                state['weight_norm'] = weight_norm
                state['adam_norm'] = adam_norm
                state['trust_ratio'] = trust_ratio
                if self.adam:
                    trust_ratio = 1

                p.data.add_(adam_step, alpha=-step_size * trust_ratio)

        return loss

In [ ]:
def main():
    train_df = pd.read_csv(f"{config.INPUT}/train.csv")
    test_df = pd.read_csv(f"{config.INPUT}/test.csv")
    sub_df = pd.read_csv(f"{config.INPUT}/sample_submission.csv")
    oof = np.zeros(len(train_df))
    test_preds_lst = []

    gkf = GroupKFold(n_splits=config.N_FOLD).split(train_df, train_df.pressure, groups=train_df.breath_id)
    for fold, (_, valid_idx) in enumerate(gkf):
        train_df.loc[valid_idx, 'fold'] = fold

    train_df['C_cate'] = train_df['C'].map({10: 0, 20: 1, 50:2})
    train_df['R_cate'] = train_df['R'].map({5: 0, 20: 1, 50:2})
    test_df['C_cate'] = test_df['C'].map({10: 0, 20: 1, 50:2})
    test_df['R_cate'] = test_df['R'].map({5: 0, 20: 1, 50:2})

    test_df['pressure'] = -1
    test_dset = VentilatorDataset(test_df)
    test_loader = DataLoader(test_dset, batch_size=config.BS,
                             pin_memory=True, shuffle=False, drop_last=False, num_workers=os.cpu_count())
    
    for fold in range(config.N_FOLD):
        if fold in []
        print(f'Fold-{fold}')
        train_dset = VentilatorDataset(train_df.query(f"fold!={fold}"))
        valid_dset = VentilatorDataset(train_df.query(f"fold=={fold}"))

        set_seed()
        train_loader = DataLoader(train_dset, batch_size=config.BS,
                                  pin_memory=True, shuffle=True, drop_last=True, num_workers=os.cpu_count(),
                                  worker_init_fn=lambda x: set_seed())
        valid_loader = DataLoader(valid_dset, batch_size=config.BS,
                                  pin_memory=True, shuffle=False, drop_last=False, num_workers=os.cpu_count())

        model = VentilatorModel()
        model.to(device)

        optimizer = AdamW(model.parameters(), lr=config.LR, weight_decay=config.WEIGHT_DECAY)

        uniqe_exp_name = f"{config.EXP_NAME}_f{fold}"
        wandb.init(project='Ventilator', entity='trtd56', name=uniqe_exp_name, group=config.EXP_NAME)
        wandb_config = wandb.config
        wandb_config.fold = fold
        for k, v in dict(vars(config)).items():
            if k[:2] == "__" or k in config.NOT_WATCH_PARAM:
                continue
            wandb_config[k] = v
        wandb.watch(model)
        
        os.makedirs(f'{config.OUTPUT}/{config.EXP_NAME}', exist_ok=True)
        model_path = f"{config.OUTPUT}/{config.EXP_NAME}/ventilator_f{fold}_best_model.bin"
        
        valid_best_loss = float('inf')
        for epoch in tqdm(range(config.N_EPOCHS)):

            train_loss, lrs = train_loop(model, optimizer, train_loader)
            valid_loss, valid_predict = valid_loop(model, valid_loader)
            valid_score = np.abs(valid_predict - train_df.query(f"fold=={fold}")['pressure'].values).mean()

            if valid_loss < valid_best_loss:
                valid_best_loss = valid_loss
                torch.save(model.state_dict(), model_path)
                oof[train_df.query(f"fold=={fold}").index.values] = valid_predict

            wandb.log({
                "train_loss": train_loss,
                "valid_loss": valid_loss,
                "valid_best_loss": valid_best_loss,
                "valid_score": valid_score,
                "learning_rate": lrs,
            })
            
            torch.cuda.empty_cache()
            gc.collect()
        
        model.load_state_dict(torch.load(model_path))
        test_preds = test_loop(model, test_loader)
        test_preds_lst.append(test_preds)
        
        sub_df['pressure'] = test_preds
        sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/sub_f{fold}.csv", index=None)
        
    train_df['oof'] = oof
    train_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/oof.csv", index=None)
    
    if len(config.SKIP_FOLDS) == 0:
        sub_df['pressure'] = np.stack(test_preds_lst).mean(0)
        sub_df.to_csv(f"{config.OUTPUT}/{config.EXP_NAME}/submission.csv", index=None)
    
        cv_score = train_df.apply(lambda x: abs(x['oof'] - x['pressure']), axis=1).mean()
        print("CV:", cv_score)

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
wandb.finish()